In [6]:
# %load notebook_setup.py
#Use %load notebook_setup.py to load these commands at the top of your
#ipython notebook.  Includes running the std_libraries.py file. 

%run std_libraries.py
%load_ext autoreload
%autoreload 2
# %matplotlib 
# plt.ioff()
#%matplotlib inline
%matplotlib notebook

what computer are you on? a = Ben's laptop, b = gpucluster, c = Ben's desktop, d = othera
base directory is C:\Users\BMH_work\github\yeast_esr_expression_analysis
Importing yeast_esr_exp and setting base_dir and data_processing_dir
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Assigns synteny groups to post WGH species using YGOB

In [7]:
# #Load GOI ohnolog data
# fname = data_processing_dir + os.sep + os.path.normpath('kl_sc_PKA_as_m24_r1g1_20181017/gois_20181205.pkl')

# with open(fname, 'rb') as file:
#     [ohnologs_goi,ohnologs_expression_sorted,goi_criteria] = pickle.load(file)

#ohnologs_expression_sorted = pd.read_csv(data_processing_dir + os.path.normpath("kl_sc_PKA_as_m24_r1g1_20181017/20200607_ohnologs_expression_sorted.csv"), index_col=0)
ohnologs_dePKA = pd.read_csv(data_processing_dir + os.path.normpath("kl_sc_PKA_as_m24_r1g1_20181017/20200617_ohnologs_dePKA.csv"), index_col=0)



In [8]:
#Extract files using url
window = '8'

pillars_goi_fname = os.path.normpath(data_processing_dir + "ortholog_files_YGOB/YGOB_pillars_goi.txt")

with open(pillars_goi_fname,'w') as f:
    for anc in ohnologs_dePKA.index:

        YGOB_query_url = 'http://ygob.ucd.ie/cgi/browser/tab.pl?ver=Latest&win=' + window + '&gene=' + anc
        YGOB_query_response = requests.get(YGOB_query_url)

        soup = BeautifulSoup(YGOB_query_response.text)
        table = soup.find("table")

        output_rows = []
        for table_row in table.findAll('tr'):
            columns = table_row.findAll('td')
            output_row = []
            for column in columns:
                output_row.append(column.text)
            output_rows.append(output_row)

        synteny_window = pd.DataFrame(output_rows[2:],columns = output_rows[1])
        synteny_row = synteny_window[synteny_window['Ancestor']==anc]
        print("\t".join(synteny_row.iloc[0,:].values) + '\n')
        f.write("\t".join(synteny_row.iloc[0,:].values) + '\n')

Kpol_1016.27	TPHA0I02760	TBLA0D05340	NDAI0B04970	NCAS0B07620	KNAG0C01610	KAFR0A03570	CAGL0G07667g	Suva_11.245	Skud_11.246	Smik_11.270	YKR013W	Kwal_14.2409	KLTH0H09812g	SAKL0D05984g	Ecym_6326	AAL178W	KLLA0D02420g	TDEL0D01590	---	Anc_1.291	YJL079C	Smik_10.163	Skud_10.134	Suva_6.151	CAGL0F05137g	KAFR0A01580	KNAG0B01940	NCAS0A09350	---	TBLA0C02750	TPHA0I01070	Kpol_282.1

Kpol_487.13	TPHA0O00680	TBLA0D04410	NDAI0G00810	---	---	KAFR0H03200	---	Suva_14.333	Skud_14.314	Smik_14.315	YNL014W	Kwal_0.167	KLTH0H11902g	SAKL0D08316g	Ecym_1312	AAL028W	KLLA0F12210g	TDEL0B06150	ZYRO0E05148g	Anc_1.391	YLR249W	Smik_12.322	Skud_12.330	Suva_10.346	CAGL0B03487g	KAFR0I02110	KNAG0I01340	NCAS0A08760	NDAI0G05180	---	---	Kpol_1048.72

Kpol_1014.7	---	TBLA0C05230	---	---	---	KAFR0I02360	CAGL0B03663g	Suva_14.350	Skud_14.327	Smik_14.328	YNR001C	Kwal_0.299	KLTH0H12606g	SAKL0D09152g	Ecym_1280	AAR004C	KLLA0F12760g	TDEL0B05830	ZYRO0E04290g	Anc_1.422	YCR005C	Smik_3.85	Skud_3.66	Suva_3.25	CAGL0H03993g	KAFR0H03050	KNAG0I011

Kpol_370.7	TPHA0O00450	TBLA0D03860	NDAI0B01230	NCAS0A13010	KNAG0I00280	KAFR0I02790	CAGL0I01386g	Suva_12.133	Skud_10.267	Smik_10.327	YJR049C	Kwal_27.10143	KLTH0C03322g	SAKL0D11902g	Ecym_3568	AFR361C	KLLA0F16885g	TDEL0C02550	ZYRO0F05302g	Anc_1.484	YEL041W	Smik_5.52	Skud_5.49	Suva_5.28	CAGL0L01265g	KAFR0E00490	KNAG0G00470	NCAS0G04160	NDAI0G00290	TBLA0B02150	TPHA0F03350	Kpol_1059.17

Kpol_1054.49	TPHA0O01070	TBLA0D03720	NDAI0D01460	NCAS0G03430	---	KAFR0E00320	CAGL0H09130g	Suva_11.21	Skud_11.19	Smik_11.23	YKL201C	Kwal_27.9992	KLTH0C02618g	SAKL0D12672g	Ecym_3157	---	KLLA0B13794g	---	ZYRO0G01826g	Anc_1.509	YJR061W	Smik_10.351	Skud_10.285	Suva_12.150	CAGL0H01793g	KAFR0D03640	KNAG0B00240	NCAS0A12220	NDAI0A04020	TBLA0B01980	TPHA0F03540	Kpol_1056.19

Kpol_529.20	---	TBLA0B00220	---	NCAS0D00140	KNAG0E02930	KAFR0F00150	CAGL0C05577g	Suva_4.5	Skud_4.15	Smik_4.2	YDL239C	Kwal_23.4424	KLTH0B08316g	SAKL0E14212g	Ecym_6476	ABL193C	KLLA0D19580g	TDEL0E00510	ZYRO0F15378g	Anc_2.16	YNL225C	Smik_14.107	Skud_14.1

Kpol_1017.6	TPHA0D03280	TBLA0D02960	NDAI0B05880	NCAS0F03560	KNAG0A07950	KAFR0G03700	CAGL0L08448g	Suva_16.478	Skud_16.444	Smik_16.402	YPR149W	Kwal_55.21238	KLTH0F14850g	SAKL0F02750g	Ecym_1236	AFR312W	KLLA0D16280g	TDEL0D05610	ZYRO0D09966g	Anc_3.493	YGR131W	Smik_6.227	Skud_7.442	Suva_7.419	CAGL0I10494g	KAFR0C01970	KNAG0B00800	NCAS0E00800	NDAI0G00930	TBLA0C04490	---	---

Kpol_480.6	TPHA0A05740	TBLA0C04540	---	NCAS0F03630	---	KAFR0G03760	---	Suva_16.485	Skud_16.451	Smik_16.409	YPR157W	Kwal_47.18937	KLTH0G02222g	SAKL0F02398g	Ecym_1225	AFR324W	KLLA0E03785g	TDEL0D05730	ZYRO0G19602g	Anc_3.505	YGR141W	Smik_6.236	Skud_7.451	Suva_7.428	CAGL0I10296g	KAFR0C02050	KNAG0B00710	NCAS0E00720	NDAI0I02810	---	---	---

Kpol_195.2	TPHA0A05810	TBLA0I03270	NDAI0B06010	NCAS0F03730	KNAG0E01590	KAFR0I02380	CAGL0A03696g	Suva_5.123	Skud_5.136	Smik_5.150	YER027C	Kwal_47.19040	KLTH0G01848g	SAKL0F02002g	Ecym_1210	AER361C	KLLA0B00583g	TDEL0D05850	ZYRO0E08404g	Anc_3.517	YGL208W	Smik_7.55	Skud_7.60	Suva_7.53	CAGL0K09350g	

Kpol_495.12	TPHA0C00450	TBLA0J00770	NDAI0F00940	NCAS0G00850	KNAG0B03240	KAFR0C05840	CAGL0H08261g	Suva_8.72	Skud_15.184	Smik_15.196	YOR019W	Kwal_47.18246	KLTH0G05632g	SAKL0G06886g	Ecym_5591	AEL233C	KLLA0F02684g	TDEL0A04750	ZYRO0A05478g	Anc_5.609	YDR475C	Smik_4.752	Skud_4.749	Suva_2.649	CAGL0M08954g	KAFR0H00640	KNAG0C06150	NCAS0A08100	NDAI0H00960	---	TPHA0D02830	Kpol_1018.64

Kpol_495.17	TPHA0D02900	TBLA0E03280	---	NCAS0A08030	KNAG0C06210	KAFR0H00580	CAGL0M08800g	Suva_2.425	Skud_4.521	Smik_4.506	YDR259C	Kwal_55.19947	KLTH0E02222g	SAKL0G07128g	Ecym_5580	AEL222C	KLLA0E16875g	---	---	Anc_5.619	YOR028C	Smik_15.206	Skud_15.193	Suva_8.81	---	KAFR0C05900	KNAG0B03190	NCAS0G00800	NDAI0F00900	TBLA0J00790	TPHA0C00500	---

Kpol_303.4	TPHA0J01310	TBLA0J00940	NDAI0H00650	NCAS0C05440	---	KAFR0H00440	CAGL0G01969g	Suva_11.307	Skud_11.308	Smik_11.334	YKR072C	Kwal_YGOB_55.21677	KLTH0F17314g	SAKL0G01166g	Ecym_5478	AAR140W	KLLA0F16258g	TDEL0B02680	ZYRO0B15862g	Anc_5.655	YOR054C	Smik_15.231	Skud_15.216	Suva_8

Kpol_1018.91	TPHA0E00910	---	---	---	---	---	---	Suva_5.215	Skud_5.219	Smik_5.240	YER096W	Kwal_27.10865	KLTH0C06820g	SAKL0F12276g	Ecym_4689	ACR227W	KLLA0E20263g	TDEL0C01610	ZYRO0B03212g	Anc_7.386	YBL061C	Smik_2.54	Skud_2.42	Suva_2.55	CAGL0A04411g	KAFR0L01700	KNAG0B02390	NCAS0E02650	NDAI0E04290	TBLA0C05190	TPHA0A00930	Kpol_479.24

Kpol_1018.93	TPHA0E00870	---	NDAI0A01670	NCAS0A14370	KNAG0C03580	KAFR0K01810	CAGL0I05522g	Suva_5.217	Skud_5.223	Smik_5.244	YER098W	Kwal_27.10842	KLTH0C06732g	SAKL0F12430g	Ecym_4685	AGR370W	KLLA0E20351g	TDEL0C01570	ZYRO0B03300g	Anc_7.390	YBL067C	Smik_2.49	Skud_2.38	Suva_2.50	CAGL0A04477g	KAFR0L01730	KNAG0B02370	NCAS0E02610	NDAI0E04180	TBLA0D00620	TPHA0P00680	Kpol_1045.41

Kpol_1045.20	---	---	---	---	KNAG0C03380	---	CAGL0G03619g	Suva_5.240	Skud_5.260	Smik_5.264	YER119C	Kwal_27.10702	KLTH0C06006g	SAKL0F13134g	Ecym_7133	AGL285C	KLLA0E06931g	TDEL0C02820	ZYRO0B04026g	Anc_7.420	YBL089W	Smik_2.31	Skud_2.19	Suva_2.28	CAGL0H04587g	KAFR0L01910	KNAG0B02190	NCAS0E02440	ND

In [10]:
#Build ortholog mapping from S.Cer to post WGH species for gois
spec_order_post_WGH = ['Scer','Smik','Skud','Suva', 'Cgla', 'Kafr','Knag','Ndai','Ncas','Tbla','Tpha','Vpol'] 

spec1 = 'Scer'
for spec2 in list(set(spec_order_post_WGH)-set(spec1)):
    yeast_esr_exp.write_YGOB_orth_lookup_table_goi(spec1,spec2)

C:\Users\BMH_work\github\yeast_esr_expression_analysis\expression_data\ortholog_files_YGOB\Scer-Ndai-goi-orthologs.txt
C:\Users\BMH_work\github\yeast_esr_expression_analysis\expression_data\ortholog_files_YGOB\Scer-Tbla-goi-orthologs.txt
C:\Users\BMH_work\github\yeast_esr_expression_analysis\expression_data\ortholog_files_YGOB\Scer-Cgla-goi-orthologs.txt
C:\Users\BMH_work\github\yeast_esr_expression_analysis\expression_data\ortholog_files_YGOB\Scer-Vpol-goi-orthologs.txt
C:\Users\BMH_work\github\yeast_esr_expression_analysis\expression_data\ortholog_files_YGOB\Scer-Kafr-goi-orthologs.txt
C:\Users\BMH_work\github\yeast_esr_expression_analysis\expression_data\ortholog_files_YGOB\Scer-Ncas-goi-orthologs.txt
C:\Users\BMH_work\github\yeast_esr_expression_analysis\expression_data\ortholog_files_YGOB\Scer-Tpha-goi-orthologs.txt
C:\Users\BMH_work\github\yeast_esr_expression_analysis\expression_data\ortholog_files_YGOB\Scer-Scer-goi-orthologs.txt
C:\Users\BMH_work\github\yeast_esr_expression_an